# Цепочка обязанностей (Chain of responsibility)

## Реализация самой простой цепочки обязанностей

Нам необходимо описать персонажа:

In [1]:
class Character:
    def __init__(self):
        self.name = "Nagibator"
        self.xp = 0
        self.passed_quests = set()
        self.taken_quests = set()

Реализуем несколько квестов

In [2]:
# Список констант дял квестов
QUEST_SPEAK, QUEST_HUNT, QUEST_CARRY = "QSPEAK", "QHUNT", "QCARRY"

In [3]:
class Event:
    def __init__(self, kind):
        self.kind = kind # тип события

Теперь начнем писать цепочку с нулевого обработчика

In [4]:
class NullHandler:
    def __init__(self, successor=None):
        self.__successor = successor
    
    def handle(self, char, event):
        """Сам обработчик"""
        if self.__successor is not None: # Если следующий обработчик вообще существует, передадим ему это событие
            self.__successor.handle(char, event)

Теперь настало время описать наши квесты в виде цепочки обязанностей

In [5]:
class QuestSpeak(NullHandler):
    def handle(self, char, event):
        if event.kind == QUEST_SPEAK:
            quest_name = "Поговорить с фермером"
            xp = 100
            if quest_name not in (char.passed_quests | char.taken_quests):
                print(f"Квест получен: \"{quest_name}\"")
                char.taken_quests.add(quest_name)
            elif quest_name in char.taken_quests:
                print(f"Квест сдан: \"{quest_name}\"")
                char.passed_quests.add(quest_name)
                char.taken_quests.remove(quest_name)
                char.xp += xp
        else:
            print("Передаю событие дальше")
            super().handle(char, event)

In [6]:
class QuestHunt(NullHandler):
    def handle(self, char, event):
        if event.kind == QUEST_HUNT:
            quest_name = "Охота на крыс"
            xp = 300
            if quest_name not in (char.passed_quests | char.taken_quests):
                print(f"Квест получен: \"{quest_name}\"")
                char.taken_quests.add(quest_name)
            elif quest_name in char.taken_quests:
                print(f"Квест сдан: \"{quest_name}\"")
                char.passed_quests.add(quest_name)
                char.taken_quests.remove(quest_name)
                char.xp += xp
        else:
            print("Передаю событие дальше")
            super().handle(char, event)

In [7]:
class QuestCarry(NullHandler):
    def handle(self, char, event):
        if event.kind == QUEST_CARRY:
            quest_name = "Принести доски из сарая"
            xp = 200
            if quest_name not in (char.passed_quests | char.taken_quests):
                print(f"Квест получен: \"{quest_name}\"")
                char.taken_quests.add(quest_name)
            elif quest_name in char.taken_quests:
                print(f"Квест сдан: \"{quest_name}\"")
                char.passed_quests.add(quest_name)
                char.taken_quests.remove(quest_name)
                char.xp += xp
        else:
            print("Передаю событие дальше")
            super().handle(char, event)

Теперь напишем класс, который раздает квесты

In [8]:
class QuestGiver:
    def __init__(self):
        self.handlers = QuestCarry(QuestHunt(QuestSpeak(NullHandler))) # Сама цепочка
        self.events = []
        
    def add_event(self, event):
        self.events.append(event)
        
    def handle_quests(self, character):
        for event in self.events:
            self.handlers.handle(character, event)

Для начала создадим список всех доступных квестов

In [9]:
events = [Event(QUEST_CARRY), Event(QUEST_HUNT), Event(QUEST_SPEAK)]

Теперь создаим квест-гивера

In [10]:
quest_giver = QuestGiver()

Добавим ему квестов

In [11]:
for event in events:
    quest_giver.add_event(event)

Теперь создадим нашего игрока

In [12]:
player = Character()

In [13]:
quest_giver.handle_quests(player)

Квест получен: "Принести доски из сарая"
Передаю событие дальше
Квест получен: "Охота на крыс"
Передаю событие дальше
Передаю событие дальше
Квест получен: "Поговорить с фермером"


In [14]:
print(player.taken_quests)
print(player.passed_quests)

{'Принести доски из сарая', 'Охота на крыс', 'Поговорить с фермером'}
set()


In [15]:
quest_giver.handle_quests(player)

Квест сдан: "Принести доски из сарая"
Передаю событие дальше
Квест сдан: "Охота на крыс"
Передаю событие дальше
Передаю событие дальше
Квест сдан: "Поговорить с фермером"


In [16]:
quest_giver.handle_quests(player)

Передаю событие дальше
Передаю событие дальше
Передаю событие дальше


In [18]:
print(player.taken_quests)
print(player.passed_quests)

set()
{'Принести доски из сарая', 'Охота на крыс', 'Поговорить с фермером'}
